## PSAT data

### PDF2Text

In [1]:
import os
from PyPDF2 import PdfReader
import re
import pprint

file_path = os.path.join(os.getcwd(), 'Data')
file_name = os.listdir(file_path)
file_name

['LEET_2021_L.hwp', 'LEET_2021_L(E).hwp', 'PSAT_2023.pdf']

In [2]:
def clean_psat_text(file_name):
    psat_text = []
    for file in file_name:
        if file.endswith('.pdf'):
            pdf = PdfReader(os.path.join(file_path, file))
            text = ''
            
            for i, page in enumerate(pdf.pages):
                text += page.extract_text()
            
            text = text.replace(' .', '.')
            text = text.replace(' , ', ', ')
            text = text.replace(' ? ', '? ')
            text = re.sub(r"(\d+)\.", r"\1. ", text)
            text = re.sub(r"(①)|(②)|(③)|(④)|(⑤)", r"\1\2\3\4\5 ", text)

            psat_text.append(text)

    return psat_text

In [3]:
psat_text = clean_psat_text(file_name)

In [4]:
pprint.pprint(psat_text[0][18000:20000])

('를 파견한다.\n'
 '③ C를 파견하 지 않으면 D를 파견하 지 않는다.\n'
 '④ C를 파견하 지 않으면 E를 파견하 지 않는다.\n'
 '⑤ D나 E를 파견하 면 C를 파견한다.16. 다음 글의 내용이 참일 때 반드시 참인 것은?\n'
 '  영어 회화가 가능한 갑순과 을돌, 중국어 회화가 가능한 병수와 \n'
 '정희를 다음 <배치 원칙>에 따라 총무부, 인사부, 영업부, \n'
 '자재부에 각 한 명씩 모두 배치하기로 하였다. 네 명 중 병수를 \n'
 '제외한 나머지는 신입사원이고, 갑순만 공인노무사 자격증을 \n'
 '갖고 있다.\n'
 '<배치 원칙>\n'
 '○총무부와 인사부 중 한 곳에는 공인노무사 자격증을 갖고 \n'
 '있는 사원을 배치한다.\n'
 '○영업부와 자재부 중 한 곳에만 중국어 회화 가능자를 배치\n'
 '한다.\n'
 '○정희를 인사부에도 자재부에도 배치하지 않는다면, 영업부에 \n'
 '배치한다.\n'
 '○영업부와 자재부 중 한 곳에만 신입사원을 배치한다.\n'
 '  이 원칙에 따라 부서를 배치한 결과 일부 사원의 부서만 \n'
 '결정되었다. 이에 다음의 원칙을 추가하였다.\n'
 '<추가 원칙>\n'
 '○인사부와 영업부에 같은 외국어 회화를 할 수 있는 사원들을 \n'
 '배치한다.\n'
 '  그 결과 <배치 원칙>을 어기지 않으면서 위 네 명의 배치를 \n'
 '다 결정할 수 있었다.\n'
 '① <배치 원칙>만으로 배치된 갑순의 부서는 영업부이다.\n'
 '② <배치 원칙>만으로 배치된 을돌의 부서는 자재부이다.\n'
 '③ <배치 원칙>과 <추가 원칙>에 따라 최종적으로 배치된 병수의 \n'
 '부서는 자재부이다.\n'
 '④ <배치 원칙>과 <추가 원칙>에 따라 최종적으로 배치된 정희의 \n'
 '부서는 인사부이다.\n'
 '⑤ <배치 원칙>과 <추가 원칙>에 따라 최종적으로 배치된 갑순의 \n'
 '부서도 을돌의 부서도 총무부가 아니다.2023년도 국가공무원 5급 공채 등 필기시험 언어논리영역 가책

- '2023년도 국가공무원 5급 공채 등 필기시험 언어논리영역 가책형 n쪽' 삭제
- `text = re.sub(r"(\d+)\.", r"\1. ", text)`: 1~9만 가능, 여기 조건에 숫자 더 늘려야함

- 7번: \ue0ac -> π

## LEET data

### Hwp2Text

In [5]:
import olefile
import zlib
import struct

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")
        
        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()
	
    # 파일 불러오기 
    def load(self, filename):
        return olefile.OleFileIO(filename)
	
    # hwp 파일인지 확인 header가 없으면 hwp가 아닌 것으로 판단하여 진행 안함
    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

	# 문서 포맷 압축 여부를 확인
    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

	# bodytext의 section들 목록을 저장
    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]
	
    # text를 뽑아내는 함수
    def get_text(self):
        return self.text

	# 전체 text 추출
    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

	# section 내 text 추출
    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self.is_compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                text += rec_data.decode('utf-16')
                text += "\n"

            i += 4 + rec_len

        return text
        
# text 추출 함수 -> 이 함수를 사용하면 됨
def get_text(filename):
    hwp = HWPExtractor(filename) 
    return hwp.get_text()

In [6]:
import pprint
import re

def clean_text(text):
    # Replace the old Chinese word with Korean word
    text = text.replace("氠瑢", "빈칸")
    text = text.replace("敤敱", "π")
    
    # Remove specific sequences
    sequences_to_remove = ["\t", "\x00", "\x02", "\x10", "\x15", "\x0b", "\x0c", "Ȁ", "Ȱ", "צ", "ȶ", "ɞ", "ض", "ᧀ", "ˇ", "ᣣ", "ᣜ", "ฒ", "ᛩ", "빈칸\r\n"]
    for seq in sequences_to_remove:
        text = text.replace(seq, "")
    text = text.replace("Ā", " ")
    
    # Remove Chinese characters
    text = re.sub(r'[\u4e00-\u9fff]+', '', text)
    
    # Define a regex pattern:
    pattern = re.compile("[^가-힣0-9a-zA-Z!?#:<>()[]{};π∞'\".,\-~ \n①-⑮㉠-㉮△○~]")
    cleaned_text = pattern.sub("", text)
    
    cleand_text = cleaned_text.replace("\x1f", "") # 물결 앞뒤고 \x1f가 붙어있는 경우가 있음(미해결)
    
    return cleaned_text

In [15]:
len(hwp_text), len(clean_leet_text)

(33760, 33207)

In [16]:
hwp_text = get_text(os.path.join(file_path, 'LEET_2021_L(E).hwp'))
pprint.pprint(hwp_text[32000:])

('이라는 한계 속에서 이루어지는 것임을 부정할 수 없기 때문이다. 뻔히 부적절한 결과가 예상되는 경우에도 문언에 구속될 것을 요구하는 것은 '
 '일견 합리적이지 않아 보일 수 있다. 그럼에도 불구하고 문언을 강조하는 입장은 ‘재량’이 연상시키는 ‘사람의 지배’에 대한 우려와, '
 '민주주의의 본질에 대한 성찰을 배경으로 하는 것임을 이해할 필요가 있다. 법률은 시민의 대표들이 지난한 타협의 과정 끝에 도출해 낸 '
 '결과물이다. 엄밀히 말해 오로지 법률의 문언 그 자체만이 민주적으로 결정된 것이며, 그 너머의 것에 대해서는, 심지어 입법 의도나 법률의 '
 '목적이라 해도 동등한 권위를 인정할 수 없다. 이러한 입장에서는 법률 적용의 결과가 부적절한지 여부보다 그것이 부적절하다고 결정할 수 '
 '있는 권한을 특정인에게 부여할 것인지 여부가 더 중요한 문제일 수 있다. 요컨대 해석자에게 그러한 권한을 부여하는 것이 바람직하지 않다고 '
 '생각하는 한, 비록 부적절한 결과가 예상되는 경우라 하더라도 여전히 문언에 구속될 것을 요구하는 편이 오히려 합리적일 수도 있는 '
 '것이다.\r\n'
 '28.윗글과 일치하는 것은?\r\n'
 '①\x1e'
 '전통적인 법학방법론 학설의 입장에서는 결국 문언을 넘은 해석과 문언에 반하는 해석을 구별하지 않는다.\r\n'
 '②\x1e'
 '종래의 법철학 학설 중 의미의 중심부와 주변부의 구별을 강조하는 입장에서는 해석에 있어 법률의 목적보다 문언에 주목한다.\r\n'
 '③\x1e'
 '민주주의의 본질을 강조하는 입장에서는 비록 법률의 적용에 따른 것이라도 실질적으로 부적절한 결과를 인정할 수는 없다고 본다.\r\n'
 '④\x1e'
 '법률 적용 결과의 합당성을 강조하는 입장에서는 문언이 제공하는 답이 부적절한지 여부는 해석자의 주관에 따라 달라질 수 있다고 '
 '주장한다.\r\n'
 '⑤\x1e'
 '법학방법론과 법철학의 논의를 하나의 연결된 구성으로 제시하는 입장에서는 언어적 불확정성으로 인해 법률이 부적절

In [14]:
clean_leet_text = clean_text(hwp_text)
pprint.pprint(clean_leet_text[32000:])

('이다.\r\n'
 '28.윗글과 일치하는 것은?\r\n'
 '①\x1e'
 '전통적인 법학방법론 학설의 입장에서는 결국 문언을 넘은 해석과 문언에 반하는 해석을 구별하지 않는다.\r\n'
 '②\x1e'
 '종래의 법철학 학설 중 의미의 중심부와 주변부의 구별을 강조하는 입장에서는 해석에 있어 법률의 목적보다 문언에 주목한다.\r\n'
 '③\x1e'
 '민주주의의 본질을 강조하는 입장에서는 비록 법률의 적용에 따른 것이라도 실질적으로 부적절한 결과를 인정할 수는 없다고 본다.\r\n'
 '④\x1e'
 '법률 적용 결과의 합당성을 강조하는 입장에서는 문언이 제공하는 답이 부적절한지 여부는 해석자의 주관에 따라 달라질 수 있다고 '
 '주장한다.\r\n'
 '⑤\x1e'
 '법학방법론과 법철학의 논의를 하나의 연결된 구성으로 제시하는 입장에서는 언어적 불확정성으로 인해 법률이 부적절한 답을 제공하는 사안에 '
 '주목한다.\r\n'
 '29.에 대한 진술로 가장 적절한 것은?\r\n'
 '판단하기 어려운 사안\r\n'
 '①\x1e'
 '법률의 문언이 극도로 명확한 경우에는 판단하기 어려운 사안이 발생하지 않는다.\r\n'
 '②\x1e'
 '판단하기 어려운 사안의 해석을 위해 법률의 목적에 구속되어야 하는 것은 아니다.\r\n'
 '③\x1e'
 '문언을 넘은 해석은 문언이 해석자를 전혀 이끌어 주지 못할 때 비로소 시도될 수 있다.\r\n'
 '④\x1e'
 '문언에 반하는 해석은 법률의 흠결이 있을 때 이를 보충하기 위한 것인 한 정당화될 수 있다.\r\n'
 '⑤\x1e'
 '형식상 드러나 있는 법률의 흠결을 보충하기 위해서도 해당 법률의 본래적 구상보다는 전체 법질서를 고려한 해석이 필요하다.\r\n'
 '\r\n'
 '30.[A]의 입장에서 ㉠을 해석한 것으로 가장 적절한 것은?\r\n'
 '①\x1e'
 '규칙의 목적이 야생의 생물 다양성을 보존하기 위한 것이라면, 멸종 위기 품종의 길고양이를 입양하는 것이 허용될 것이다.\r\n'
 '②\x1

### Hwp2PDF

In [ ]:
import win32com.client as win32
import win32gui

os.chdir(file_path)

hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
hwnd = win32gui.FindWindow(None, "빈 문서 1 - 한글")

win32gui.ShowWindow(hwnd, 0)
hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

for i in file_name:
    hwp.Open(os.path.join(file_path, i))  # 한/글로 열어서
    hwp.HAction.GetDefault('FileSaveAsPdf', hwp.HParameterSet.HFileOpenSave.HSet)  # PDF로 저장할 건데, 설정값은 아래와 같이.
    hwp.HParameterSet.HFileOpenSave.filename = os.path.join(file_path, i.replace('.hwp', 'pdf'))  # 확장자는 .pdf로,
    hwp.HParameterSet.HFileOpenSave.Format = 'PDF'  # 포맷은 PDF로,
    hwp.HAction.Execute('FileSaveAsPdf', hwp.HParameterSet.HFileOpenSave.HSet) 
    
win32gui.ShowWindow(hwnd, 5)
hwp.XHwpDocuments.Close(isDirty=False)
hwp.Quit()

del hwp
del win32

### PDF2Text